<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#EmoInt-dataset-for-emotion-detection-in-lyrics" data-toc-modified-id="EmoInt-dataset-for-emotion-detection-in-lyrics-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>EmoInt dataset for emotion detection in lyrics</a></span><ul class="toc-item"><li><span><a href="#EmoInt-statistics" data-toc-modified-id="EmoInt-statistics-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>EmoInt statistics</a></span></li></ul></li><li><span><a href="#Modeling" data-toc-modified-id="Modeling-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Modeling</a></span></li><li><span><a href="#References" data-toc-modified-id="References-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>References</a></span></li></ul></div>

# EmoInt dataset for emotion detection in lyrics

Existing emotion datasets are mainly annotated categorically without an indication of degree of emotion. EmoInt, instead, provides several tweets annotated according to an emotion (anger, fear, joy, sadness) and to the degree at which the emotion is expressed in text.

It is important to mention that EmoInt was manually annotated, using [Best-Worst Scaling](https://nparc.nrc-cnrc.gc.ca/eng/view/fulltext/?id=b132b0af-2ae0-4964-ac3a-493e7292a37a) (BWS), an annotation scheme shown to obtain very reliable scores.

For our purpose, we will consider each tweet to be like a lyric and, on top of that, we will perform our feature engineering using spaCy and the other tools we used so far.

Our original dataset, MoodyLyrics, contains "happy", "sad", "angry" and "relaxed" as labels. Therefore, in order to perform a sort of interjection with EmoInt, we will just use the tweets corresponding to the anger, joy and sadness emotions.

The remaining part of this notebook assumes that we have already parsed EmoInt dataset in a .csv file which we can use to train some machine learning models as we did when we performed our feature engineering on lyrics. For more information about how this .csv was generated, please refer to the `src/emoint_parser.py` script.

## EmoInt statistics

As EmoInt provide intensity levels together with emotion labels, we decided to take into account only those tweets for which the intensity was greater that 0.50 (50%). Also, we dropped tags and hashtags (e.g. "Hey @MrTwitter how are you? #cool" became "Hey how are you?") because we will have to compare those tweets with songs and songs do not have those kind of things.

# Modeling

# References
[EmoInt paper](http://saifmohammad.com/WebDocs/TweetEmotionIntensities-starsem2017.pdf)